### Topic Modeling 

In [9]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim import corpora, models
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
import matplotlib.pyplot as plt
import pandas as pd

In [26]:
# Download stopwords
nltk.download("punkt")
nltk.download("stopwords")
stop_words = set(stopwords.words("italian"))

[nltk_data] Downloading package punkt to /Users/ziade/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/ziade/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2. Text Processing 

In [11]:
def preprocess_text(text):
    tokens = word_tokenize(text.lower())  # Tokenize and convert to lowercase
    tokens = [word for word in tokens if word.isalnum() and word not in stop_words]  # Remove stopwords and punctuation
    return tokens


In [12]:
def train_lda(documents, num_topics=3):
    processed_docs = [preprocess_text(doc) for doc in documents]
    dictionary = corpora.Dictionary(processed_docs)
    corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
    lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)
    return lda_model, corpus, dictionary


In [22]:
import os 

dr = '/Users/ziade/Desktop/DataScience/Siiam_projects/llm_finetuning'
os.listdir(dr)


['sample_2017.xlsx',
 '.DS_Store',
 'data_cleaning.ipynb',
 'dataset',
 '.env',
 'sample_2017.csv',
 '.gradio',
 '.ipynb_checkpoints',
 'ssm17 soluzioni.pdf']

In [37]:
df = pd.read_csv(f'{dr}/sample_2017.csv')
df.head()

,questionID,testyear,numberintest,picture_provided,question_text,picture_link,option_a,option_b,option_c,option_d,option_e,correct_option,correct_fulltext
0,IT0001,2017,1,No,"[Una donna di 27 anni, asintomatica, in buone ...",NaN,Non vi è indicazione all'avvio immediato di al...,Avviare terapia con clopidogrel,Avviare terapia con warfarin,Avviare terapia con eparina a basso peso molec...,Avviare terapia con prednisone a basse dosi,a,Non vi è indicazione all'avvio immediato di al...
1,IT0002,2017,2,No,Con quale termine si definisce la presenza di ...,NaN,Coristoma,Amartoma,Differenziazione anomala,Tiloma,Cordoma,a,Coristoma
2,IT0003,2017,3,No,Quale delle seguenti è una traslocazione tipic...,NaN,t(11;22),t(14;18),t(8;14),t(9;22),t(11;18),a,t(11;22)
3,IT0004,2017,4,No,"[ Una donna di 35 anni, nullipara, si reca dal...",NaN,HPV,HIV,Neisseria gonorrhoeae,HSV-1,HSV-2,a,HPV
4,IT0005,2017,5,No,Come viene standardizzata attualmente la lettu...,NaN,Sistema di Bethesda,Sistema di Papanicolau,Linee guida di Tokio,Criteri di Glasgow,Livelli di Berg,a,Sistema di Bethesda


In [32]:
# preprocess the text
#df['question_text'] = df['question_text'].apply(preprocess_text)

df['question_text'] = df['question_text'].apply(lambda x: ' '.join(x))

In [40]:
text_list = df['question_text'].tolist()
single_text = ' '.join(text_list)

In [33]:
df['question_text'] = df['question_text'].apply(preprocess_text)


In [16]:
sample_text = """ 
**Artificial Intelligence (AI): A Mid-Level Overview**

Artificial Intelligence (AI) is a branch of computer science that focuses on creating systems capable of performing tasks that typically require human intelligence. These tasks include learning from data, recognizing patterns, understanding natural language, solving problems, and making decisions. AI is driven by various technologies, including machine learning, deep learning, neural networks, and natural language processing.

One of the key components of AI is machine learning, which enables computers to improve their performance by analyzing large amounts of data without explicit programming. Deep learning, a subset of machine learning, utilizes artificial neural networks to process complex data, making it particularly effective for image and speech recognition.

AI is widely used in multiple fields, such as finance, cybersecurity, manufacturing, transportation, and entertainment. Autonomous vehicles, recommendation systems, virtual assistants, and fraud detection systems all rely on AI technologies. Ethical concerns regarding AI include issues related to data privacy, bias in decision-making, and the impact of automation on employment.

Researchers and developers continue to refine AI models, aiming to enhance their efficiency, accuracy, and adaptability. As AI advances, it holds the potential to revolutionize industries and improve productivity in ways previously unimaginable.

---

**Understanding Healthcare: A Mid-Level Overview**

Healthcare refers to the organized efforts and systems designed to maintain or improve people’s physical and mental well-being. It encompasses a wide range of services, including disease prevention, diagnosis, treatment, rehabilitation, and health education. Healthcare is provided by medical professionals such as doctors, nurses, pharmacists, and therapists across various settings, including hospitals, clinics, and long-term care facilities.

The healthcare system consists of three primary components: primary, secondary, and tertiary care. **Primary care** is the first level of contact for patients, dealing with general health concerns, preventive measures, and routine check-ups. **Secondary care** includes specialized services such as cardiology, dermatology, and orthopedics, often provided by hospitals and specialists. **Tertiary care** involves highly specialized medical treatments, such as organ transplants and cancer therapies, typically available in advanced medical centers.

Public health initiatives play a crucial role in healthcare by promoting wellness, controlling disease outbreaks, and ensuring access to vaccinations. Governments, private organizations, and international institutions work together to improve healthcare infrastructure and policies. Key challenges in healthcare include accessibility, affordability, medical research advancements, and managing chronic diseases like diabetes and heart disease.

As technology evolves, healthcare is increasingly integrating digital solutions, such as electronic health records (EHRs), telemedicine, and personalized medicine, to enhance efficiency and patient outcomes. Ensuring quality care while addressing global health disparities remains a significant priority for healthcare professionals and policymakers.



"""

In [41]:
lda_model, corpus, dictionary = train_lda([single_text], num_topics=5)

# Show topics
for i, topic in lda_model.show_topics(num_topics=5, formatted=True):
    print(f"Topic {i+1}: {topic}")


Topic 1: 0.001*"anni" + 0.001*"paziente" + 0.001*"seguenti" + 0.001*"viene" + 0.001*"dolore" + 0.001*"diagnosi" + 0.001*"soccorso" + 0.001*"uomo" + 0.001*"pronto" + 0.001*"qual"
Topic 2: 0.001*"anni" + 0.001*"paziente" + 0.001*"viene" + 0.001*"seguenti" + 0.001*"diagnosi" + 0.001*"presenta" + 0.001*"soccorso" + 0.001*"uomo" + 0.001*"pronto" + 0.001*"giunge"
Topic 3: 0.002*"anni" + 0.001*"viene" + 0.001*"paziente" + 0.001*"seguenti" + 0.001*"diagnosi" + 0.001*"uomo" + 0.001*"presenta" + 0.001*"terapia" + 0.001*"qual" + 0.001*"giunge"
Topic 4: 0.001*"anni" + 0.001*"viene" + 0.001*"paziente" + 0.001*"seguenti" + 0.001*"diagnosi" + 0.001*"soccorso" + 0.001*"sinistro" + 0.001*"uomo" + 0.001*"pronto" + 0.001*"terapia"
Topic 5: 0.026*"anni" + 0.015*"paziente" + 0.013*"viene" + 0.012*"seguenti" + 0.007*"diagnosi" + 0.007*"uomo" + 0.006*"presenta" + 0.006*"pronto" + 0.006*"qual" + 0.006*"soccorso"


In [42]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

vis = gensimvis.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis)


In [46]:
# Assuming lda_model is already trained and available
topics = lda_model.print_topics(num_words=10)  # Adjust the number of words as needed

# Display each topic with its top words
for topic in topics:
    print(f"Topic {topic[0]}: {topic[1]}")


Topic 0: 0.001*"anni" + 0.001*"paziente" + 0.001*"seguenti" + 0.001*"viene" + 0.001*"dolore" + 0.001*"diagnosi" + 0.001*"soccorso" + 0.001*"uomo" + 0.001*"pronto" + 0.001*"qual"
Topic 1: 0.001*"anni" + 0.001*"paziente" + 0.001*"viene" + 0.001*"seguenti" + 0.001*"diagnosi" + 0.001*"presenta" + 0.001*"soccorso" + 0.001*"uomo" + 0.001*"pronto" + 0.001*"giunge"
Topic 2: 0.002*"anni" + 0.001*"viene" + 0.001*"paziente" + 0.001*"seguenti" + 0.001*"diagnosi" + 0.001*"uomo" + 0.001*"presenta" + 0.001*"terapia" + 0.001*"qual" + 0.001*"giunge"
Topic 3: 0.001*"anni" + 0.001*"viene" + 0.001*"paziente" + 0.001*"seguenti" + 0.001*"diagnosi" + 0.001*"soccorso" + 0.001*"sinistro" + 0.001*"uomo" + 0.001*"pronto" + 0.001*"terapia"
Topic 4: 0.026*"anni" + 0.015*"paziente" + 0.013*"viene" + 0.012*"seguenti" + 0.007*"diagnosi" + 0.007*"uomo" + 0.006*"presenta" + 0.006*"pronto" + 0.006*"qual" + 0.006*"soccorso"
